In [9]:
from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification

: 

: 

In [ ]:
us_equity_news=pd.read_csv("/Users/williammacpherson/Documents/FinancialDatasets/us_equities_news_dataset.csv")
us_equity_news.dropna()
us_equity_news=us_equity_news.drop(['provider','url','article_id','id','category'], axis=1)

analyst_ratings=pd.read_csv("/Users/williammacpherson/Documents/FinancialDatasets/Daily Financial News/raw_analyst_ratings.csv")
analyst_ratings.dropna()
analyst_ratings=analyst_ratings.drop(['Unnamed: 0','url','publisher'], axis=1)

In [ ]:
analyst_ratings.head()

,headline,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:59-04:00,A


In [ ]:
analyst_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407328 entries, 0 to 1407327
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   headline  1407328 non-null  object
 1   date      1407328 non-null  object
 2   stock     1407328 non-null  object
dtypes: object(3)
memory usage: 32.2+ MB


In [ ]:
us_equity_news.head()

,ticker,title,content,release_date
0,NIO,Why Shares of Chinese Electric Car Maker NIO A...,What s happening\nShares of Chinese electric c...,2020-01-15
1,NIO,NIO only consumer gainer Workhorse Group amon...,Gainers NIO NYSE NIO 7 \nLosers MGP Ingr...,2020-01-18
2,NIO,NIO leads consumer gainers Beyond Meat and Ma...,Gainers NIO NYSE NIO 14 Village Farms In...,2020-01-15
3,NIO,NIO NVAX among premarket gainers,Cemtrex NASDAQ CETX 85 after FY results \n...,2020-01-15
4,NIO,PLUG NIO among premarket gainers,aTyr Pharma NASDAQ LIFE 63 on Kyorin Pharm...,2020-01-06


In [ ]:
us_equity_news=us_equity_news.rename(columns={'release_date':'date','ticker':'stock','title':'headline'})
us_equity_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221513 entries, 0 to 221512
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   stock     221513 non-null  object
 1   headline  221513 non-null  object
 2   content   221505 non-null  object
 3   date      221513 non-null  object
dtypes: object(4)
memory usage: 6.8+ MB


In [ ]:
df_array = np.array(us_equity_news)
df_list = list(df_array[:,1])


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
#tokenize text to be sent to model
inputs = tokenizer(df_list, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

model.config.id2label


positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':df_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df2 = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

df2.head()